###### Importing required files 

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from collections import Counter
from scipy.sparse import csr_matrix
import math 
import re


###### Importing and Reading data using Pandas from csv (comma seperated values) Here it is tab seperated values

In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv',delimiter ='\t')
target = df['is_duplicate']
print(target.head(5))
print(df.describe())
print(df[:5])


0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64
                  id           qid1           qid2   is_duplicate
count  168354.000000  168354.000000  168354.000000  168354.000000
mean    84176.500000  115204.941421  115633.193758       0.372245
std     48599.757947   76343.737500   76745.727760       0.483405
min         0.000000       1.000000       2.000000       0.000000
25%     42088.250000   46894.500000   46649.500000       0.000000
50%     84176.500000  107798.000000  108209.500000       0.000000
75%    126264.750000  179709.500000  180905.750000       1.000000
max    168353.000000  260809.000000  260810.000000       1.000000
   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? 

###### Converting the rows of DataFrame into list which contains sentences as each instance

In [3]:
l_1 = df['question1'].astype(str).tolist()
l_2 = df['question2'].astype(str).tolist()



###### Removing the grammar ex: '?' etc from the data set

In [4]:
def noGrammer(li):
    l = li
    for i in range(len(l)):
        l[i] = re.sub('[^a-zA-Z0-9\']', ' ', l[i])
    return l


In [5]:
list_1 = noGrammer(l_1)
list_2 = noGrammer(l_2)


###### Converting each sentence into list of words using split() function

In [6]:
text = [l.split() for l in list_1]
text1 =[l.split() for l in list_2]

print (len(text), len(text1), type(text), type(text1))

168354 168354 <class 'list'> <class 'list'>


###### Appending both the lists to create a combined csr matrix

In [7]:
text2 = text + text1
print (len(text2))

336708


###### Filtering words using minimum word length as measure

In [8]:
#def filterLen(docs, minlen): 
#    r""" filter out terms that are too short. docs is a list of lists, each inner list is a document represented as a list of words minlen is the minimum length of the word to keep """ 
#    return [ [t for t in d if len(t) >= minlen ] for d in docs ] 
#docs_final = filterLen(text2, 2)
docs_final = text2

###### Building CSR(Compressed Sparse Rows) matrix where each word is represented as feature

In [9]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [10]:
mat = build_matrix(docs_final)
csr_info(mat)


 [nrows 336708, ncols 76288, nnz 3569803]


###### Converting the CSR marix into inverse document frequency matrix to make it easy for calculation

In [11]:

# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)
print("mat1:", mat[15,:20].todense(), "\n")
print("mat2:", mat2[15,:20].todense(), "\n")
print("mat3:", mat3[15,:20].todense())

mat1: [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]] 

mat2: [[ 1.03005906  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]] 

mat3: [[ 0.04924313  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]]


In [12]:
mat3.shape

(336708, 76288)

###### Dividing the matrix into Query matrix 1 and Query matrix 2

In [13]:
query_mat1 = mat3[:168354, :]
query_mat2 = mat3[168354:336708, :]

In [14]:
query_mat2.shape

(168354, 76288)

In [15]:
query_mat1.shape

(168354, 76288)

In [16]:

rows = query_mat1.shape[0]
columns = query_mat1.shape[1]
rows
columns

76288

###### Taking a sample of 405 queries to find out the cosine similarity

In [32]:
result = []
s=0
rows1 = 405 #math.ceil(rows/10000)
column1 = columns
print(rows1, column1)
    
        


405 76288


##### Cosine Similarity without dimensionality reduction

In [33]:
for i in range(rows1):
    for j in range(column1):
        s+=(query_mat1[i,j] * query_mat2[i,j])
    result.append(s)
    s=0


###### Sample Result of the above similarity

In [34]:
print(result[0:10])

[0.95457300276469914, 0.63548022048394803, 0.13078660827503402, 0.0, 0.24097646710963014, 0.43739767675365537, 0.0, 0.80849080877150792, 0.97973797735255241, 0.60445559582886643]


In [35]:
class_test = []
for i in range(0,rows1):
    if result[i]>0.55:
        class_test.append(1) 
    else:
         class_test.append(0)
        


In [36]:
accuracy = []
for i in range(0,rows1):
    if class_test[i] == target[i]:
        accuracy.append(1)
    else:
        accuracy.append(0)


In [37]:
np.mean(accuracy)

0.64197530864197527